In [1]:
from pybullet_environment import *
from guide_previous import IntersectionVolumeGuide as guide1
from guide import IntersectionVolumeGuide as guide2

device = "cuda" if torch.cuda.is_available() else "cpu"

pybullet build time: Jul 17 2023 13:38:55


In [2]:
env = RobotEnvironment(manipulator = True, gui = False)
guide_seq = guide1(env)

argv[0]=


In [3]:
obstacle_config = np.zeros((1, 10))

# Obstacle A:
obs_pos = np.array([0.6, 0., 0.18])
obs_quat = np.array(env.client_id.getQuaternionFromEuler(np.array([0., 0., 0.])))
obs_size = np.array([0.5, 0.4, 0.36])
obstacle_config[0] = np.array([*obs_pos, *obs_quat, *obs_size])

# Obstacle B:
# obs_pos = np.array([0.7, 0., 0.6])
# obs_quat = np.array(env.client_id.getQuaternionFromEuler(np.array([0., 0., 0.])))
# obs_size = np.array([0.3, 0.12, 1.2])
# obstacle_config[1] = np.array([*obs_pos, *obs_quat, *obs_size])

env.spawn_obstacles(obstacle_config)

In [19]:
env.clear_obstacles()

In [4]:
guide_parallel = guide2(env, obstacle_config, device)

In [5]:
joint_config = np.zeros((9,))[:-2]
joint_config[0] = 0.11
joint_config[1] = 0.913
joint_config[2] = 0.12
joint_config[3] = -0.96
joint_config[4] = 0.23
joint_config[5] = 1.43
joint_config[6] = 0.3

joint_config = torch.tensor(joint_config[np.newaxis, :, np.newaxis], dtype = torch.float32, device = device)

print(guide_parallel.cost2(joint_config))

obs_pose = np.array([*obs_pos, *obs_quat])
print(guide_seq.cost(joint_config[0, :, 0], obs_pose, obs_size))

Link  torch.Size([1, 1, 9, 4, 4])
tensor(0.0107, device='cuda:0')
tensor(0.0107)


In [5]:
joint_config = np.zeros((9,))[:-2]
joint_config[0] = -0.371
joint_config[1] = 0.522
joint_config[2] = -0.134
joint_config[3] = -1.4
joint_config[4] = 0.341
joint_config[5] = 1.7
joint_config[6] = 0.572

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [13]:
for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, start_joints[i])

In [11]:
goal_joints = np.array([0.9798614, 0.5573511, -0.1629493, -1.9912002, -0.23168434, 2.214461, -0.38091224, 0., 0.])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, goal_joints[i])

In [22]:
obs_pos = np.array([0.7, 0., 0.6])
obs_ori = np.array([0., 0., 0.])
obs_quat = np.array(env.client_id.getQuaternionFromEuler(obs_ori))
obs_pose = np.array([*obs_pos, *obs_quat])

obs_size = np.array([0.3, 0.12, 1.2])

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = obs_size/2,
                                       rgbaColor = np.hstack([env.colors['yellow'], np.array([1.0])]))

obs_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = obs_pos, 
                                       baseOrientation = obs_quat)

: 

In [21]:
env.client_id.removeBody(obs_id)

In [8]:
joint_config = np.zeros((9,))[:-2]
joint_config[0] = 0.11
joint_config[1] = 0.913
joint_config[2] = 0.12
joint_config[3] = -0.96
joint_config[4] = 0.23
joint_config[5] = 1.43
joint_config[6] = 0.3

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [10]:
joint_tensor = torch.tensor(joint_config, dtype = torch.float32)
joint_tensor.requires_grad = True

In [12]:
for _ in range(100):

    joint_tensor.grad = None
    cost = guide.cost(joint_tensor, obs_pose, obs_size)

    cost.backward()

    with torch.no_grad():
        joint_tensor -= 0.2 * joint_tensor.grad

    # joint_config = joint_tensor.detach().numpy()
    # for i, joint_ind in enumerate(env.joints):
    #     env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

: 

In [9]:
print(joint_tensor.grad)

tensor([-0.0462,  0.0962, -0.0306, -0.0592, -0.0029, -0.0081, -0.0001])


: 